In [45]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, ParameterGrid, train_test_split
from sklearn import metrics

import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier

# # import warnings filter
# from warnings import simplefilter
# # ignore all future warnings
# simplefilter(action='ignore', category=FutureWarning)

In [46]:
# dataset_path = 'allUsers.lcl.csv'
dataset_path = "../data/tommaso_data/allUsers.lcl.csv"

df = pd.read_csv(dataset_path)
#knn Cannot work with mining data, so replace them with a 0
df = df.replace('?', 0)
#df = df.dropna(thresh=len(df) * 0.10, axis=1)



In [47]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score


def save_results(clf, X_test, y_test, nclf, dataf):
    dataf.loc[nclf, 'acc'] = accuracy_score(y_test, clf.predict(X_test))
    dataf.loc[nclf,'prec'] = precision_score(y_test, clf.predict(X_test), average='weighted')
    dataf.loc[nclf,'recall'] = recall_score(y_test, clf.predict(X_test), average='weighted')
    # dataf.loc[nclf, 'test f1 score (0)'] = f1_score(y_test, clf.predict(X_test), pos_label=0)

    # dataf.loc[nclf, 'test f1 score (1)'] = f1_score(y_test, clf.predict(X_test), pos_label=1)
    dataf.loc[nclf, 'test f1 score (W)'] = f1_score(y_test, clf.predict(X_test), average='macro')
    # dataf.loc[nclf,'ROC AUC'] = roc_auc_score(y_test, clf.predict_proba(X_test)[:,1], multi_class='ovr')
    return dataf

results_df = pd.DataFrame()



# KNN

# Class recognition with User label

In [48]:
df2 = df.drop(['Class'], axis=1)
labels_Class = df['Class']


X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Class, test_size=0.20, random_state=42)


In [49]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=1)

In [50]:
print(knn.score(X_test, y_test))


0.9602432778489116


In [51]:
results_df = save_results(knn, X_test, y_test, 'KNN', results_df)



# Hand owner recognition with Class label

In [52]:
df2 = df.drop(['User'], axis=1)
labels_Users = df['User']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=0.20, random_state=42)

In [53]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=1)

In [54]:
print(knn.score(X_test, y_test))

0.8646606914212548


In [55]:
results_df = save_results(knn, X_test, y_test, 'KNN', results_df)



# Class recognition without User label

In [56]:
df2 = df.drop(['User'], axis=1)
df2 = df2.drop(['Class'], axis=1)
labels_Class = df['Class']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Class, test_size=0.20, random_state=42)

In [57]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
print(knn.score(X_test, y_test))

0.9596670934699104


In [58]:
results_df = save_results(knn, X_test, y_test, 'KNN', results_df)



# Class recognition without Class label

In [59]:
df2 = df.drop(['User'], axis=1)
df2 = df2.drop(['Class'], axis=1)

labels_Users = df['User']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=0.20, random_state=42)

In [60]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
print(knn.score(X_test, y_test))

0.8646606914212548


In [61]:
results_df = save_results(knn, X_test, y_test, 'KNN', results_df)



# SVM

In [62]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# !pip insall sklearn.grid_search
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix


# dataset_path = 'allUsers.lcl.csv'

df = pd.read_csv(dataset_path)

df = df.replace('?', np.nan)
df = df.dropna(thresh=len(df) * 0.8 , axis=1)
df = df.replace(np.nan, 0)

drop_indices = np.random.choice(df.index, 75000, replace=False)
df = df.drop(drop_indices)
df

,Class,User,X0,Y0,Z0,X1,Y1,Z1,X2,Y2,Z2,X3,Y3,Z3,X4,Y4,Z4,X5,Y5,Z5
4,1,0,55.329647,71.707275,-63.688956,36.561863,81.868749,-52.752784,86.320630,68.214645,-72.228461,61.5961571288978,11.2506481750465,-68.9564252307431,77.3872254123912,42.7178334810919,-72.0151462991019,0,0,0
60,1,0,55.845720,66.522803,-68.410839,77.106544,36.734284,-75.072331,85.470310,62.473837,-77.605075,34.2457979030067,76.2252873173598,-61.5592919042357,62.4769143002536,5.03622819257662,-68.6089267801289,0,0,0
93,1,0,59.676152,10.346545,-70.922560,32.903327,80.877307,-56.737537,84.383038,67.579617,-74.432037,75.716530692884,42.4039470792251,-73.8682413989786,0,0,0,0,0,0
103,1,0,60.643217,10.291196,-70.175276,35.024652,81.172235,-54.580198,84.337446,67.490598,-74.242394,76.4325165255543,42.4279202681865,-72.79701886308,60.0147927287663,71.7732083115917,-59.384569022977,0,0,0
111,1,0,57.520111,71.715919,-61.757361,34.581397,81.222350,-55.219979,59.636048,10.242790,-71.191428,76.9848900528974,42.536531586955,-72.3894288430609,85.0430132473624,67.8348313672923,-73.6765462237176,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78040,5,14,61.739726,100.844190,-6.994374,-1.605232,97.814815,2.909392,26.067348,136.421557,-32.985276,-21.9183977994588,103.586732377954,-55.7435172836229,28.2762399952328,107.284601079317,9.04071794397873,-30.8317317238542,78.1007664596381,-16.6639696803137
78045,5,14,-29.873947,76.914756,-18.651672,-2.029566,123.468663,-45.304928,-1.565969,97.739744,3.231634,61.6464047727335,100.690969420769,-7.00426363012687,26.0991865935174,136.304277368217,-33.2657491003375,-22.2004005073228,103.169253119125,-55.957281522383
78058,5,14,55.477667,129.359734,-43.070249,78.681362,31.968211,-45.536851,-2.214866,125.482435,-40.814235,29.5598097604374,106.099892829171,8.60633476048323,61.6908493868336,101.361080251551,-4.50175708504207,-1.33922415636317,98.0202263508904,4.31976810449226
78064,5,14,88.760663,59.931435,-63.671039,-2.155721,125.068618,-41.709071,77.992760,32.445566,-44.458812,-0.970094534882875,97.628680322975,2.98397838329817,55.1306108005195,129.367411656344,-43.3191719272339,-30.185045982206,78.1287444666287,-16.3108442369111


# Class recognition without User label

In [63]:
df2 = df.drop(['User'], axis=1)
df2 = df2.drop(['Class'], axis=1)
labels_Class = df['Class']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Class, test_size=0.20, random_state=42)

clf = SVC(kernel='rbf', gamma="auto", C = 7)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.25161290322580643


In [64]:

results_df = save_results(clf, X_test, y_test, 'SVC', results_df)


C:\Users\polcg\.virtualenvs\MD2-NovaiZ3s\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# SVC gread search class recognition without user label

In [65]:
param_grid = {'C':[1,10,100,1000],
              'gamma':[1,0.1,0.001,0.0001], 
              'kernel':['sigmoid','rbf']}

grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2)

grid.fit(X_train,y_train)


Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.1s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.1s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.1s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.1s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.1s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.6s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.6s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.6s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.6s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.6s
[CV] END .....................C=1, gamma=0.1, kernel=sigmoid; total time=   0.1s
[CV] END .....................C=1, gamma=0.1, k

KeyboardInterrupt: 

In [ ]:
grid.best_params_
predic = grid.predict(X_test)
print(grid.score(X_test, y_test))
print(grid.best_params_)

In [ ]:

results_df = save_results(grid, X_test, y_test, 'SVC', results_df)



# Hand owner recognition without Class label


In [ ]:
df2 = df.drop(['User'], axis=1)
df2 = df2.drop(['Class'], axis=1)
labels_Users = df['User']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=0.20, random_state=42)

clf = SVC(kernel='rbf', gamma="auto", C = 7)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

In [ ]:

results_df = save_results(clf, X_test, y_test, 'SVC', results_df)



# Gread SearchHand owner recognition without Class label

In [ ]:
param_grid = {'C':[1,10,100,1000],
              'gamma':[1,0.1,0.001,0.0001], 
              'kernel':['sigmoid','rbf']}

grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2)

grid.fit(X_train,y_train)

In [ ]:
grid.best_params_
predic = grid.predict(X_test)
print(grid.score(X_test, y_test))
print(grid.best_params_)

In [ ]:
results_df = save_results(clf, X_test, y_test, 'SVC', results_df)



# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
# dataset_path = 'allUsers.lcl.csv'

df = pd.read_csv(dataset_path)
#knn Cannot work with mining data, so replace them with a 0
df = df.replace('?', 0)
#df = df.dropna(thresh=len(df) * 0.10, axis=1)


# Class recognition with User label

In [ ]:
df2 = df.drop(['User'], axis=1)
labels_Users = df['User']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=0.20, random_state=42)

In [ ]:
clf = RandomForestClassifier(max_depth=200, random_state=10)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
results_df = save_results(clf, X_test, y_test, 'Random forest', results_df)



# Gread Search

## Show results:

In [66]:
results_df.sort_values(by=['acc'], ascending=False)



,acc,prec,recall,test f1 score (W)
KNN,0.864661,0.863969,0.864661,0.796469
SVC,0.251613,0.660048,0.251613,0.127833
